<img src = "https://github.com/singlestore-labs/spaces-notebooks/blob/e551e274bb67bb1e5081131ee1150cdba713fc43/common/images/singlestore-jupyter.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Data Loader</h1>
    </div>
</div>

In [11]:
!pip cache purge --quiet

In [12]:
import pandas as pd
from singlestoredb import notebook as nb

In [13]:
df = pd.read_csv(
        nb.stage.open("Final_Dataset.csv", "r")
    )

In [14]:
df.head()

,Fillings,Fraud
0,nanitem 14 exhibits financial statements repor...,yes
1,item 14 principal accounting fees services mat...,no
2,item 14 exhibits financial statements schedule...,yes
3,item 14 exhibits financial statement schedules...,yes
4,item 14 exhibits financial statement schedules...,no


In [15]:
df.rename(columns = {"Fillings": "Filings"}, inplace = True)

In [16]:
%%sql
DROP DATABASE IF EXISTS fraud_detection_demo;
CREATE DATABASE IF NOT EXISTS fraud_detection_demo;

++
||
++
++

In [17]:
%%sql
USE fraud_detection_demo;

DROP TABLE IF EXISTS financial_filings;
CREATE TABLE financial_filings (
    id BIGINT AUTO_INCREMENT NOT NULL,
    Filings LONGTEXT,
    Fraud VARCHAR(3),
    PRIMARY KEY(id)
);

++
||
++
++

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [23]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

In [25]:
df.to_sql(
    "financial_filings",
    con = db_connection,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

170

In [26]:
new_df = pd.read_sql(
    "SELECT Fraud, Filings FROM financial_filings LIMIT 10",
    con = db_connection
)

In [27]:
new_df

,Fraud,Filings
0,no,item 14 exhibits financial statements financia...
1,no,item 14 exhibits financial statement schedules...
2,yes,item 14 exhibits financial statement schedules...
3,yes,item 14 exhibits financial statement schedules...
4,no,item 14 exhibits financial statement schedules...
5,no,item 14 principal accountant fees services pur...
6,yes,item 14 exhibits financial statement schedules...
7,yes,item 14 exhibits financial statement schedules...
8,no,item 14 principal accountant fees services inf...
9,yes,item 14 exhibits financial statement schedules...
